# Linear Algebra

## Introduction
Blah, Blah, Blah.

## *avos* Product for Vectors

Now that we've defined the Red-Black adjacency matrix, *pedigree numbers*, the *generation* function and the *avos* product for scalers, we can begin to explore extensions to linear algebra. We'll first do so by defining the *avos* product for two vectors. 

Assuming 0 indexing in the tha matrix, the 4th row vector is an incomplete *downstream* view for $vertex_{4}$ while the 2nd column vector is an incomplete *upstream* view for $vertex_{2}$. You can see that $vertex_{4}$ is related to $vertex_{2}$ (by *pedigree number* 5). We would like to define the *avos* dot product of these two vectors to provide a scaler that is the *pedigree number* of this relationship:

\begin{equation*}
\begin{vmatrix}
2 & 0 & 0 & 0 & 1 \\
\end{vmatrix}
\cdot
\begin{vmatrix}
3 \\
0 \\
1 \\
0 \\
0 \\
\end{vmatrix}
= 5
\end{equation*}

The vector dot product, summing element-wise products, results in a scaler value of 6. We'd like to take a similar approach for the vector avos product. There are two issues:
1. The domain of the *avos* product is positive integers > 1, so an pairwise products with a 0 or a 1 are excluded and the element-wise product is considered to be zero.
2. It is possible for there to be multiple paths through the graph bewteen two nodes. If that were the case, then summing the element-wise *avos* product would not result in the *pedigree number* representing the relationship from $vertex_{a}$ to $vertex_{c}$. For the *avos* vector product, rather than summing the element-wise products we arbitrarily choos the non-zero minimum element-wise product, thus representing the "closest" relationship between $vertex_{a}$ and $vertex_{c}$.

A simple implementation of the *avos* vector product follows:

In [2]:
from redblackgraph.simple_impl import avos

def vec_avos(x, y):
    '''Given two vectors, compute the "avos" dot product.'''

    # if clause on list comprehension screens out input outside domain of avos
    # min over the list arbitrarily chooses "nearest" relationship
    try:
        return min([avos(a, b) for a, b in zip(x, y) if a > 1 and b > 1])
    except ValueError as e:
        # if there are no element-wise avos results, min will run across an empty sequence and
        # through a ValueError...
        return 0

## Matrix multiplication

With scaler and vector *avos* products defined, extension to matrices is elementary. Given *A* and *B*, both matrices following the contraints defined for Red-Black adjacency matrices, and *C* = *A* x *B*, the elements of C<sub>ij</sub> are given by:
* if i != j, the vector *avos* product of *row vector<sub>i</sub>* from A and *column vector<sub>j</sub>* from B
* if i == j and *A<sub>ij</sub>* == *B<sub>ij</sub>*, *A<sub>ij</sub>*
* otherwise, undefined.

Matrix multiplication of Red-Black adjacency matrices may seem a little abstract so let's consider a practical example to motivate the analysis. Given a Red-Black adjacency matrix, *A<sub>rb</sub>*, the result of *A<sub>rb</sub>*<sup>2</sup> shows all vertices directly related by following up to 2 relationship edges, the result of *A<sub>rb</sub>*<sup>3</sup> shows all vertices related by following up to 3 relationship edges, etc. For some *n* <= |*V*| there will be a *A<sub>rb</sub>*<sup>n</sup> == *A<sub>rb</sub><sup>+</sup>*.

## Vector Classifications

Within the context of *A<sub>rb</sub><sup>+</sup>, we will define several vector classes. These vectors each represent a vertex (red or black) not yet in the graph.

* *simple row vector* - represented by *v<sub>s</sub>*. Elements are constrained to values of 0, 2 or 3 with at most 1 element of 2 and at most 1 element of 3. The element positions that are non-zero represent a direct edge to the vertex corresponding to the element index.
* *simple column vector* - represented by *u<sub>s</sub>*. Elements are constrained to have elements with values of 0, and 2 if the vertex is *red* or 3 if the vertex is *black*. The element positions that are non-zero represent a direct edge from the vertices corresponding to the element indices.
* *closed row vector* - represented by *v<sub>c</sub>*. Elements are constrained to values of 0, and positive integers greater than 1. The element positions that are non-zero represent the transitively closed relationships between the vertex represented by this vector and the vertices in the graph.
* *closed column vector* - represented by *u<sub>c</sub>*. Elements are constrained to values of 0, and even positive integers greater than 1 if the vertex is red or odd positive integers greater than 1 if the vertex is black. The element positions that are non-zero represent the transitively closed relationships from the vertices in the graph to this vector.

With these definitions, the following is observed:

1. *v<sub>s</sub>* x *A<sub>rb</sub><sup>+</sup> = *v<sub>c</sub>*
2. *A<sub>rb</sub><sup>+</sup> x *u<sub>s</sub>* = *u<sub>c</sub>*

## Relational Composition

The question arise how to correctly update *A<sub>rb</sub><sup>+</sup>* with the addition of new vertices to the graph. We propose an algorithm that is relatively simple in concept that generates *A<sup>'</sup><sub>rb</sub><sup>+</sup>*. Consider adding a vertex to the graph corresponding to *A<sub>rb</sub><sup>+</sup>* of size *N*. We define a relational composition that can be used to add a new node to the graph and produce *A<sup>'</sup><sub>rb</sub><sup>+</sup>* of size *N* + 1. Conceptually:

*A<sup>'</sup><sub>rb</sub><sup>+</sup>* = *v* *A<sub>rb</sub><sup>+</sup>* *u*

Precisely this relational composition is accomplished by:

1. generate *v<sub>c</sub>* (given by observation 1 above)
2. generate *u<sub>c</sub>* (given by observation 2 above)
3. *A<sup>'</sup><sub>rb</sub><sup>+</sup>* is composed by:
    1. appending *v<sub>c</sub>* as a new row
    2. appending *u<sub>c</sub>* as a new column
    3. setting element *A<sup>'</sup><sub>rb</sub><sup>+</sup>*<sub>N+1, N+1</sub> to 0 if vertex is red, and 1 if vertex is black
    4. For each row, r, where *v<sub>c<sub>i</sub></sub>* != 0, set *A<sup>'</sup><sub>rb</sub><sup>+</sup>*<sub>r,i</sub> = *v<sub>c<sub>r</sub></sub>* avos *u<sub>c<sub>i</sub></sub>*

## Areas of Further investigation

* Determinants
* Eigenvalues
* Loop prevention
* *avos* properties (commutitivity, associativity, distribution, identity, inverse, etc.)
* Spectral Graph Theory
* Explore ideas to reduce dimensionality (embedding similar to word2vec, etc.)
* Axioms of linear algebra:
** u + (v + w) = (u + v) + w
** v + w = w + v
** There is a vector *0* such that *0* + v = v for all v
** For every vector v there is a vector -v so that v + (-v) = *0*
** a(bv) = (ab)v
** 1v = v
** a(v + w) = av + aw
** (a + b)v = av + bv